# Run this code to deploy an agent

In [56]:
import vertexai

PROJECT_ID = "virtual-nurse-450613"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://deploy-short-bot"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

from google.adk.agents import Agent
from vertexai.preview.reasoning_engines import AdkApp
import yfinance as yf

def get_stock_price(ticker: str):
    stock = yf.Ticker(ticker)
    price = stock.info.get("currentPrice", "Price not available")
    return {"price": price, "ticker": ticker}

root_agent = Agent(
    model='gemini-2.0-flash-001',
    name='root_agent',
    description='A simple agent that get stock price.',
    instruction="""You are a stock price assistant. Always use the get_stock_price tool.
                Include the ticker symbol in your response""",
    tools=[get_stock_price]
)

from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

session = app.create_session(user_id="u_123")

In [57]:
session

Session(id='82097e38-a69e-4148-84ee-4210423ce442', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1750463326.5524333)

In [58]:
app.list_sessions(user_id="u_123")

ListSessionsResponse(sessions=[Session(id='82097e38-a69e-4148-84ee-4210423ce442', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1750463326.5524333)])

In [59]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="whats is Apple stock price?",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-dc50c4e4-7ff3-4c5d-8a9b-6eb8dc0ada31', 'args': {'ticker': 'AAPL'}, 'name': 'get_stock_price'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 8, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 8}], 'prompt_token_count': 71, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 71}], 'total_token_count': 79, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-2fd27fc2-8fb0-4373-a7b2-591cb49061ec', 'author': 'root_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'g2fi4c4J', 'timestamp': 1750463334.795711}
{'content': {'parts': [{'function_response': {'id': 'adk-dc50c4e4-7ff3-4c5d-8a9b-6eb8dc0ada31', 'name': 'get_stock_price', 'response': {'price': 201.0, 'ticker': 'AAPL'}}}], 'role': 'user'}, 'invocation_id': 'e-2fd27fc2-8fb0-4373-a7b2-591cb49061e

In [60]:
from vertexai import agent_engines

remote_app = agent_engines.create(
    agent_engine=root_agent,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]", "yfinance"   
    ]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'pydantic': '2.11.7', 'google-cloud-aiplatform': '1.97.0', 'cloudpickle': '3.1.1'}
The following requirements are missing: {'pydantic', 'cloudpickle'}
The following requirements are appended: {'pydantic==2.11.7', 'cloudpickle==3.1.1'}
The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'yfinance', 'pydantic==2.11.7', 'cloudpickle==3.1.1']
Using bucket deploy-short-bot
Wrote to gs://deploy-short-bot/agent_engine/agent_engine.pkl
Writing to gs://deploy-short-bot/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://deploy-short-bot/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/533873564462/locations/us-central1/reasoningEngines/6360041447986561024/operations/8864586927261614080
View progress and logs at https://console.cloud.google.com/logs/query?project=virtual-nurse-450613
AgentEngin

In [61]:
remote_app.resource_name

'projects/533873564462/locations/us-central1/reasoningEngines/6360041447986561024'

In [62]:
remote_session = remote_app.create_session(user_id="u_456")
remote_session

{'id': '8217517359288549376',
 'lastUpdateTime': 1750463547.922375,
 'userId': 'u_456',
 'state': {},
 'events': [],
 'appName': '6360041447986561024'}

In [63]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="what is Apple stock price?",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-0640b0dd-df85-407d-b95c-be73cda20fe0', 'args': {'ticker': 'AAPL'}, 'name': 'get_stock_price'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 8, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 8}], 'prompt_token_count': 71, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 71}], 'total_token_count': 79, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-4335e899-0093-4465-985e-dbae1d321a6e', 'author': 'root_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': 'uPRN0XlV', 'timestamp': 1750463568.919436}
{'content': {'parts': [{'function_response': {'id': 'adk-0640b0dd-df85-407d-b95c-be73cda20fe0', 'name': 'get_stock_price', 'response': {'price': 201.0, 'ticker': 'AAPL'}}}], 'role': 'user'}, 'invocation_id': 'e-4335e899-0093-4465-985e-dbae1d321a6e', 'author': 'root_agent', 'actions': {'state_delta': {}, 'artifact_del